In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
import gym
import numpy as np
import random
import onnx
import onnxruntime
from gym import wrappers
import os
from typing import Any, Dict, List, Optional
import glob
# from base.rl.ppo import PPO

pthfile = 'pth/ckpt.0.pth'
onnxpath = 'onnx/ckpt.85.onnx'
print(torch.__version__)
print(onnx.__version__)
print(onnxruntime.__version__)

1.2.0
1.6.0
1.6.0


In [21]:
class Flatten(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return x.view(x.shape[0], -1)

class Global_Actor(nn.Module):
    def __init__(self):
        super().__init__()
        self.actor = nn.Sequential(  # (8, G, G)
            nn.Conv2d(8, 8, 3, padding=1),  # (8, G, G)
            nn.BatchNorm2d(8),
            nn.ReLU(),
            nn.Conv2d(8, 4, 3, padding=1),  # (4, G, G)
            nn.BatchNorm2d(4),
            nn.ReLU(),
            nn.Conv2d(4, 4, 5, padding=2),  # (4, G, G)
            nn.BatchNorm2d(4),
            nn.ReLU(),
            nn.Conv2d(4, 2, 5, padding=2),  # (2, G, G)
            nn.BatchNorm2d(2),
            nn.ReLU(),
            nn.Conv2d(2, 1, 5, padding=2),  # (1, G, G)
            Flatten(),  # (G*G, )
            nn.Sigmoid(),  # added for non-negative
        )
        
    def forward(self, inputs):
        x2 = self.actor(inputs)
        return x2

In [27]:
rl_net = Global_Actor()
rl_net.actor.load_state_dict(torch.load(pthfile, map_location='cpu'),strict=False)
rl_net.eval()

for _ in range(10):
    inputs = torch.zeros(1, 8, 240, 240)
    for i in range(240):
        for j in range(240):
            if random.randint(0,100)>=50:
                inputs[0][0][i][j] = 1
                
            if random.randint(0,100)>=50:
                inputs[0][1][i][j] = 1

            if random.randint(0,100)>=50:
                inputs[0][3][i][j] = 1
                
            if random.randint(0,100)>=50:
                inputs[0][4][i][j] = 1

            if random.randint(0,100)>=50:
                inputs[0][5][i][j] = 1
                
            if random.randint(0,100)>=50:
                inputs[0][7][i][j] = 1

    # print("see datas:::::", np.sum(np.array(inputs[0][0])==1))
    map3 = torch.zeros(1, 240, 240)
    map3[0][random.randint(0,100)][random.randint(0,100)] = 1.
    inputs[0][2] = map3
    inputs[0][6] = map3
    x = inputs
    torch_out = rl_net(x)
    print(torch_out)

see datas::::: 28924
tensor([[0.4786, 0.4793, 0.4784,  ..., 0.4752, 0.4748, 0.4741]],
       grad_fn=<SigmoidBackward>)
see datas::::: 28994
tensor([[0.4782, 0.4788, 0.4793,  ..., 0.4751, 0.4742, 0.4751]],
       grad_fn=<SigmoidBackward>)
see datas::::: 29100
tensor([[0.4778, 0.4779, 0.4785,  ..., 0.4745, 0.4746, 0.4746]],
       grad_fn=<SigmoidBackward>)
see datas::::: 29451
tensor([[0.4780, 0.4779, 0.4776,  ..., 0.4753, 0.4748, 0.4740]],
       grad_fn=<SigmoidBackward>)
see datas::::: 29076
tensor([[0.4780, 0.4777, 0.4773,  ..., 0.4731, 0.4747, 0.4750]],
       grad_fn=<SigmoidBackward>)
see datas::::: 29129
tensor([[0.4784, 0.4781, 0.4781,  ..., 0.4744, 0.4765, 0.4743]],
       grad_fn=<SigmoidBackward>)
see datas::::: 29104
tensor([[0.4780, 0.4780, 0.4780,  ..., 0.4749, 0.4751, 0.4750]],
       grad_fn=<SigmoidBackward>)
see datas::::: 28985
tensor([[0.4785, 0.4789, 0.4776,  ..., 0.4754, 0.4753, 0.4745]],
       grad_fn=<SigmoidBackward>)
see datas::::: 29184
tensor([[0.4784, 0.